<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LGBMClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.pipeline import Pipeline

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier 
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score

import time
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.tail()

,text,toxic
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0
159570,"""\nAnd ... I really don't think you understand...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [7]:
df.isna().mean()

text     0.0
toxic    0.0
dtype: float64

In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [9]:
tqdm.pandas()
df['text'] = df['text'].progress_apply(clean_text)

100%|████████████████████████████████| 159571/159571 [00:09<00:00, 17267.20it/s]


In [10]:
corpus = df['text'].values.astype('U')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Подготовим признаки и целевой признак перед обучением:

In [12]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [13]:
import spacy

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
def spacy_lemm(sentence):
    doc = nlp(sentence)
    return " ".join([token.lemma_ for token in doc])

df['lemm_text'] = df['text'].progress_apply(spacy_lemm)

100%|██████████████████████████████████| 159571/159571 [15:43<00:00, 169.05it/s]


In [14]:
df = df.drop(['text'], axis=1)

### Вывод

- 159571 строка
- Дубликатов и пропусков нет
- 2 столбца: toxic, text
- "text" - тексты твитов
- "toxic" - булевые значения, является ли данный твит токсичным или нет
- 90% твитов не токсичны
- с помощью регулярных выражений обработали текст для дальнейшего исследования

## Обучение

In [15]:
features = df.drop('toxic', axis=1)
target = df['toxic']

In [16]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, test_size = 0.3, random_state=42)

In [17]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/atiragrams/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Уменьшим количество кроссвалидаций до 3 из-за размера выборки.

In [18]:
cv_counts = 3

### LogisticRegression

In [19]:
%%time

lr_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=42, max_iter=2000))])

lr_params = {'clf__C': [10],
          'clf__class_weight': ['balanced']}

lr_grid = GridSearchCV(estimator=lr_pipe, param_grid=lr_params, cv=cv_counts, scoring='f1', n_jobs=-1, refit=False)
lr_grid.fit(features_train['lemm_text'], target_train)
lr_best_paramms = lr_grid.best_params_

CPU times: user 289 ms, sys: 345 ms, total: 634 ms
Wall time: 30.9 s


In [20]:
print(lr_best_paramms)
print(lr_grid.best_score_)

{'clf__C': 10, 'clf__class_weight': 'balanced'}
0.776919407113831


### LGBMClassifier

In [21]:
%%time

lgb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=42))])

lgb_params = {
  'clf__n_estimators': [200],
  'clf__learning_rate': [0.25],
  'clf__max_depth': [-1]}

lgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=lgb_params, cv=cv_counts, scoring='f1', n_jobs=-1, refit=False)
lgb_grid.fit(features_train['lemm_text'], target_train)
lgb_best_params = lgb_grid.best_params_

CPU times: user 214 ms, sys: 118 ms, total: 332 ms
Wall time: 1min 45s


In [22]:
print('Оптимальные параметры: ', lgb_best_params)
print(lgb_grid.best_score_)

Оптимальные параметры:  {'clf__learning_rate': 0.25, 'clf__max_depth': -1, 'clf__n_estimators': 200}
0.7735983445528626


Подобраны параметры с помощью GridSearchCV с расчетом TF-IDF

In [23]:
vectorize = TfidfVectorizer(ngram_range=(1,3),
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)

In [24]:
X_train = vectorize.fit_transform(features_train['lemm_text'])
X_test = vectorize.transform(features_test['lemm_text'])

In [25]:
lr_m = LogisticRegression(C=10, class_weight='balanced', random_state=42)
lr_m.fit(X_train, target_train)

LogisticRegression(C=10, class_weight='balanced', random_state=42)

In [26]:
lgb_m = LGBMClassifier(learning_rate=0.25, max_depth=-1, n_estimators=200, random_state = 12345)
lgb_m.fit(X_train, target_train)

LGBMClassifier(learning_rate=0.25, n_estimators=200, random_state=12345)

In [27]:
def scoring(fitted_model):
    test_pred = fitted_model.predict(X_test)
    test_f1 = f1_score(target_test, test_pred)
    
    print('F1 on test: {:.3f}'.format(test_f1))

In [28]:
scoring(lgb_m)

F1 on test: 0.782


In [29]:
scoring(lr_m)

F1 on test: 0.788


## Выводы

Логистическая регрессия выдает на тестовой выборке результат F1-score: 0.788, у LGBM: 0.782. Обе модели хороши, но все таки у логистической регрессии лучший результат.